# Develop negative log-likelihood with scaled inputs

In [ ]:
from __future__ import annotations

import numdifftools as nd
import numpy as np

from numpy.typing import ArrayLike
from scipy.fft import rfft, irfft, rfftfreq
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect
from scipy.optimize import approx_fprime
from numpy.random import default_rng

import thztools as thz
from thztools.thztools import _tdnll_scaled as tdnll_scaled

## Simulate measurements

In [ ]:
rng = np.random.default_rng(0)
n = 256
m = 64
ts = 0.05
t = np.arange(n) * ts
mu, _ = thz.thzgen(n, ts=ts, t0=n * ts / 3)
sigma = np.array([1e-5, 1e-2, 1e-3])
noise = thz.noiseamp(sigma, mu, ts) * rng.standard_normal((m, n))
x = np.array(mu + noise)
delta = np.zeros(n)
alpha = np.zeros(m - 1)
eta = np.zeros(m - 1)

logv = np.log(sigma**2)
scale_logv = 1e-1 * np.ones(3)
scale_delta = 1e-1 * thz.noiseamp(sigma, mu, ts)
scale_alpha = 1e-1 * np.ones(m - 1)
scale_eta = 1e-2 * np.ones(m - 1)
scale_v = 1.0e-2

## Check gradient

In [ ]:
_, grad_delta_tdnll = tdnll_scaled(
    x,
    logv,
    delta,
    alpha,
    eta,
    ts,
    fix_logv=True,
    fix_delta=False,
    fix_alpha=True,
    fix_eta=True,
    scale_logv=np.ones(3),
    scale_delta=np.ones(n),
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=1.0,
)

_, grad_delta_tdnll_scaled = tdnll_scaled(
    x,
    np.log(np.exp(logv) / scale_v) / scale_logv,
    delta / scale_delta,
    alpha,
    eta,
    ts,
    fix_logv=True,
    fix_delta=False,
    fix_alpha=True,
    fix_eta=True,
    scale_logv=scale_logv,
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=scale_v,
)

grad_delta_nd = nd.Gradient(
    lambda _delta: tdnll_scaled(
        x,
        np.log(np.exp(logv) / scale_v),
        _delta,
        alpha,
        eta,
        ts,
        fix_logv=True,
        fix_delta=True,
        fix_alpha=True,
        fix_eta=True,
        scale_logv=np.ones(3),
        scale_delta=np.ones(n),
        scale_alpha=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
        scale_v=scale_v,
    )[0],
    step=1e-6,
)(delta)

np.stack(
    (
        grad_delta_tdnll,
        grad_delta_tdnll_scaled / scale_v / scale_delta,
        grad_delta_nd / scale_v,
    )
).T

In [ ]:
plt.plot(t, grad_delta_tdnll)
plt.plot(t, grad_delta_tdnll_scaled / scale_v / scale_delta)
plt.show()
plt.plot(t, grad_delta_tdnll - grad_delta_tdnll_scaled / scale_v / scale_delta)
plt.show()

In [ ]:
val, grad_logv_tdnll = tdnll_scaled(
    x,
    logv,
    delta,
    alpha,
    eta,
    ts,
    fix_logv=False,
    fix_delta=True,
    fix_alpha=True,
    fix_eta=True,
    scale_logv=np.ones(3),
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=1.0,
)

val_scaled_v, grad_logv_tdnll_scaled_v = tdnll_scaled(
    x,
    np.log(np.exp(logv) / scale_v),
    delta,
    alpha,
    eta,
    ts,
    fix_logv=False,
    fix_delta=True,
    fix_alpha=True,
    fix_eta=True,
    scale_logv=np.ones(3),
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=scale_v,
)

val_scaled, grad_logv_tdnll_scaled = tdnll_scaled(
    x,
    np.log(np.exp(logv) / scale_v) / scale_logv,
    delta,
    alpha,
    eta,
    ts,
    fix_logv=False,
    fix_delta=True,
    fix_alpha=True,
    fix_eta=True,
    scale_logv=scale_logv,
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=scale_v,
)

grad_logv_nd = nd.Gradient(
    lambda _logv: tdnll_scaled(
        x,
        np.log(np.exp(_logv) / scale_v) / scale_logv,
        delta,
        alpha,
        eta,
        ts,
        fix_logv=True,
        fix_delta=True,
        fix_alpha=True,
        fix_eta=True,
        scale_logv=scale_logv,
        scale_delta=scale_delta,
        scale_alpha=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
        scale_v=scale_v,
    )[0]
)(logv)

print(f"{val =}")
print(f"{val_scaled_v / scale_v =}")
print(f"{val_scaled / scale_v =}")
np.stack(
    (
        grad_logv_tdnll,
        grad_logv_tdnll_scaled_v,
        grad_logv_tdnll_scaled / scale_logv,
        grad_logv_nd,
    )
).T

In [ ]:
_, grad_alpha_tdnll = tdnll_scaled(
    x,
    logv,
    delta,
    alpha,
    eta,
    ts,
    fix_logv=True,
    fix_delta=True,
    fix_alpha=False,
    fix_eta=True,
    scale_logv=np.ones(3),
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=scale_v,
)

grad_alpha_nd = nd.Gradient(
    lambda _alpha: tdnll_scaled(
        x,
        logv,
        delta,
        _alpha,
        eta,
        ts,
        fix_logv=True,
        fix_delta=True,
        fix_alpha=True,
        fix_eta=True,
        scale_logv=np.ones(3),
        scale_delta=scale_delta,
        scale_alpha=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
        scale_v=scale_v,
    )[0]
)(alpha)

np.stack((grad_alpha_tdnll, grad_alpha_nd)).T

In [ ]:
_, grad_eta_tdnll = tdnll_scaled(
    x,
    logv,
    delta,
    alpha,
    eta,
    ts,
    fix_logv=True,
    fix_delta=True,
    fix_alpha=True,
    fix_eta=False,
    scale_logv=np.ones(3),
    scale_delta=scale_delta,
    scale_alpha=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
    scale_v=scale_v,
)

grad_eta_nd = nd.Gradient(
    lambda _eta: tdnll_scaled(
        x,
        logv,
        delta,
        alpha,
        _eta,
        ts,
        fix_logv=True,
        fix_delta=True,
        fix_alpha=True,
        fix_eta=True,
        scale_logv=np.ones(3),
        scale_delta=scale_delta,
        scale_alpha=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
        scale_v=scale_v,
    )[0]
)(eta)

np.stack((grad_eta_tdnll, grad_eta_nd)).T

## Estimate noise parameters with revised NLL

In [ ]:
result = thz.tdnoisefit(x.T, v0=sigma**2, ts=ts, fix_a=False, fix_eta=False)

In [ ]:
print(result[2]["message"])

In [ ]:
var_out = result[0]["var"] * m / (m - 1)
var_err = result[2]["err"]["var"] * m / (m - 1)
for val_in, val_out, err in zip(sigma**2, var_out, var_err):
    print(f"Input: {val_in:6.4g}\t Output: {val_out:6.4g} ± {err:6.4g}")

In [ ]:
plt.plot(result[2]["grad_scaled"])
plt.show()

In [ ]:
plt.semilogy(np.diag(result[2]["hess_inv_scaled"]))
plt.show()

In [ ]:
np.diag(result[2]["hess_inv_scaled"][:3, :3])

In [ ]:
plt.plot(t, np.log10(np.diag(result[2]["hess_inv_scaled"])[3 : 3 + n]))
plt.plot(t, mu)
plt.show()

In [ ]:
plt.plot(t, result[2]["err"]["delta"] * np.sqrt(m))
plt.plot(t, thz.noiseamp(np.sqrt(result[0]["var"]), result[0]["mu"], ts))
plt.show()

## Repeat fit with amplitudes and delays fixed

In [ ]:
result = thz.tdnoisefit(x.T, v0=sigma**2, ts=ts, fix_a=True, fix_eta=True)
print(result[2]["message"])

In [ ]:
var_out = result[0]["var"] * m / (m - 1)
var_err = result[2]["err"]["var"] * m / (m - 1)
for val_in, val_out, err in zip(sigma**2, var_out, var_err):
    print(f"Input: {val_in:6.4g}\t Output: {val_out:6.4g} ± {err:6.4g}")

In [ ]:
plt.plot(t, result[2]["err"]["delta"] * np.sqrt(m))
plt.plot(t, thz.noiseamp(np.sqrt(result[0]["var"]), result[0]["mu"], ts))
plt.plot(t, np.std(x, axis=0))
plt.show()